In [160]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


commune=pd.read_parquet('donnee-comm-data.gouv-parquet-2023-geographie2024-produit-le2024-07-05.parquet')
dep=pd.read_csv('donnee-dep-data.gouv-2023-geographie2024-produit-le2024-07-05.csv',sep=';')
reg=pd.read_csv('donnee-reg-data.gouv-2023-geographie2024-produit-le2024-07-05.csv',sep=';')
commune.head(2)

,CODGEO_2024,annee,classe,unité.de.compte,valeur.publiée,faits,tauxpourmille,complementinfoval,complementinfotaux,POP,millPOP,LOG,millLOG
0,01001,16,Coups et blessures volontaires,victime,ndiff,NaN,NaN,1.698925,1.498563,767,16,348.0,16
1,01001,16,Coups et blessures volontaires intrafamiliaux,victime,diff,0.0,0.0,NaN,NaN,767,16,348.0,16


# I) Nettoyage des données dans les différents DataFrames
## A) Vérification de la cohérence et de l'uniformité des données
### 1) Vérification de la cohérence
Tout d'abord, il convient de vérifier que chaque variable est au bon type. L'objectif est d'avoir une uniformité des types de valeurs entre les 3 DataFrames commune, dep et reg. 
A noter que les dates seront traitées séparément car elles font l'objet d'une traitement particulier


In [161]:
## DataFrame commune
#les colonnes classe et unité.de.compte doivent être au format object
commune['classe']=commune['classe'].astype('str')
commune['unité.de.compte']=commune['unité.de.compte'].astype('str')
commune.info()
## DataFrame dep
dep.info()
## DataFrame reg
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3917760 entries, 0 to 3917759
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   CODGEO_2024         object 
 1   annee               int32  
 2   classe              object 
 3   unité.de.compte     object 
 4   valeur.publiée      object 
 5   faits               float64
 6   tauxpourmille       float64
 7   complementinfoval   float64
 8   complementinfotaux  float64
 9   POP                 int32  
 10  millPOP             int32  
 11  LOG                 float64
 12  millLOG             int32  
dtypes: float64(5), int32(4), object(4)
memory usage: 328.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12928 entries, 0 to 12927
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   classe            12928 non-null  object
 1   annee             12928 non-null  int64 
 2   Code.département  12928 non-nu

### 2) Vérification de l'uniformité des données
Il convient de vérifier que les données sont uniformes, c'est à dire qu'elles doivent être écrites toujours de la même manière en respectant une règle unique
#### a) Données quantitatives

In [162]:
reg.head(1)

,classe,annee,Code.région,unité.de.compte,millPOP,millLOG,faits,POP,LOG,tauxpourmille
0,Homicides,16,1,victime,16,16,32,394110,"226105,0475677566","0,08119560528786381"


In [163]:
## DataFrame commune
commune['annee'].unique() #Toutes les années sont écrites au même format
commune['classe'].unique() #Toutes les classes sont écrites au même format et il n'y en a pas deux identiques écrites différemment 
commune['unité.de.compte'].unique() #Ici victime et victime entendue correspondent a la même unité de compte. Il faut donc modifier cela 
commune['unité.de.compte']=commune['unité.de.compte'].replace('victime entendue','victime')
commune['valeur.publiée'].unique() #Toutes les valeurs publiées sont écrites au même format 

## DataFrame dep
dep['annee'].unique() #Toutes les années sont écrites au même format
dep['classe'].unique() #Toutes les années sont écrites au même format
dep['Code.département'].unique() #Tous les départements sont écrits au même format sans erreur de doublon. A noter qu'il est normal que la valeur "20" soit manquante puisqu'il s'agit de la corse, dont le code a été remplacé en 1976 par 2A et 2B
dep['unité.de.compte'].unique() #Ici victime et victime entendue correspondent a la même unité de compte. Il faut donc modifier cela 
dep['unité.de.compte']=dep['unité.de.compte'].replace('victime entendue','victime')

## DataFrame reg
reg['annee'].unique() #Toutes les années sont écrites au même format
reg['classe'].unique() #Toutes les années sont écrites au même format
reg['Code.région'].unique() #Toutes les régions sont présentes et sont écrites au même format 

array([ 1,  2,  3,  4,  6, 11, 24, 27, 28, 32, 44, 52, 53, 75, 76, 84, 93,
       94], dtype=int64)

#### b) Dates
L'étude des formats a démontré que les dates (ici "annee") sont au format int, ce qui est incorrect. Pour la suite de l'analyse, ces variables doivent être au format datetime64. 

In [165]:
commune['annee'] = pd.to_datetime(commune['annee'].apply(lambda x: f"20{x:02d}"))
dep['annee'] = pd.to_datetime(dep['annee'].apply(lambda x: f"20{x:02d}"))
reg['annee'] = pd.to_datetime(reg['annee'].apply(lambda x: f"20{x:02d}"))

,classe,annee,Code.région,unité.de.compte,millPOP,millLOG,faits,POP,LOG,tauxpourmille
0,Homicides,2016-01-01,1,victime,16,16,32,394110,"226105,0475677566","0,08119560528786381"


## B) Vérifications des doublons

In [166]:
# Recherche de doublons 
print('Il y a',commune.duplicated().sum(),'doublons dans le df commune')
print('Il y a',dep.duplicated().sum(),'doublons dans le df dep')
print('Il y a',reg.duplicated().sum(),'doublons dans le df reg')

Il y a 0 doublons dans le df commune
Il y a 0 doublons dans le df dep
Il y a 0 doublons dans le df reg


## C) Vérification des valeurs manquantes
### 1) Analyse globale

In [167]:
# Recherche de valeurs manquantes du df departement 
print('Il y a',dep.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df departement')
# Recherche de valeurs manquantes sur les lignes du df département
#dep.isna().any(axis = 1).sum() Inutile donc 

Il y a 0 colonnes contenant des valeurs manquantes dans le df departement


In [168]:
# Recherche de valeurs manquantes du df région
print('Il y a',reg.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df reg')
# Recherche de valeurs manquantes sur les lignes du df régions
#reg.isna().any(axis = 1).sum() Inutile donc 

Il y a 0 colonnes contenant des valeurs manquantes dans le df reg


In [169]:
# Recherche de valeurs manquantes sur les colonnes du df commune
print('Il y a',commune.isna().any(axis = 0).sum(),'colonnes contenant des valeurs manquantes dans le df commune') 
# Recherche de valeurs manquantes sur les lignes du df communes
print('Il y a',commune.isna().any(axis = 1).sum(),'lignes contenant des valeurs manquantes sur',commune.shape[0],'lignes sur le df.')
print('Les colonnes concernées sont les colonnes faits, taux pour mille, complementinfoval, complementinfototaux')

Il y a 4 colonnes contenant des valeurs manquantes dans le df commune
Il y a 3917760 lignes contenant des valeurs manquantes sur 3917760 lignes sur le df.
Les colonnes concernées sont les colonnes faits, taux pour mille, complementinfoval, complementinfototaux


### 2) Traitement des valeurs manquantes
Le fichier des métadonnées du dataset indique que si la variable contenue dans la colonne "valeur.publiée" vaut ndiff, alors les variables contenues dans la colonne "faits" ne sera pas renseignée. De ce fait, il a été décidé de supprimer toutes les lignes donc la colonne "valeur.publiée" vaut ndiff. 

In [170]:
# L'idée sera ici de faire une condition : si la valeur de valeur.publiee vaut ndiff, alors on supprime la ligne


## D) Analyse de la distribution des variables
Un des objectifs sous-jacents et de détecter la présence d'outliers
